In [4]:
import easyocr
reader = easyocr.Reader(['en'])
result = reader.readtext('cnic.jpeg')
for (bbox, text, prob) in result:
    print(text)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


PAKISTAN
National Identity Card
ISLAMIC REPUBLIC OF PAKISTAN
Name
Muhammad Mudassir Zaman
Ulii_}
Father Name
Shafqat Zaman
Gender
Country of Stay
M
Pakistan
Identity Number
Date of Birth
42201-2908700-1
16.04.2001
Date of Issue
Date of Expiry
20.06.2019
20.06.2029
Holder
5
Signature
Uljez


In [ ]:
import cv2
import easyocr
import face_recognition
import os
import numpy as np
import re
import matplotlib.pyplot as plt

CNIC_IMAGE = "cnic.jpeg"
KNOWN_DIR = "known_faces/dataset_imgs/"

# ============ STEP 1: OCR ============
print("🔍 Extracting text from CNIC...")

reader = easyocr.Reader(['en'])
ocr_results = reader.readtext(CNIC_IMAGE)

full_text = " ".join([text for (_, text, _) in ocr_results])
print("📝 OCR TEXT:", full_text)

# Extract CNIC Number using regex
cnic_match = re.findall(r"\d{5}-\d{7}-\d", full_text)
cnic_number = cnic_match[0] if cnic_match else "Not Found"

print("🆔 CNIC Number:", cnic_number)

# ============ STEP 2: FACE EXTRACTION ============
print("🙂 Extracting face from CNIC...")

cnic_img = face_recognition.load_image_file(CNIC_IMAGE)
face_locations = face_recognition.face_locations(cnic_img)

if len(face_locations) == 0:
    print("❌ No face found on CNIC")
    exit()

top, right, bottom, left = face_locations[0]
cnic_face = cnic_img[top:bottom, left:right]

cnic_encoding = face_recognition.face_encodings(cnic_img, face_locations)[0]

# ============ STEP 3: LOAD KNOWN DATASET ============
print("📂 Loading known faces...")

known_encodings = []
known_names = []

for file in os.listdir(KNOWN_DIR):
    path = os.path.join(KNOWN_DIR, file)
    image = face_recognition.load_image_file(path)
    encodings = face_recognition.face_encodings(image)
    
    if len(encodings) > 0:
        known_encodings.append(encodings[0])
        known_names.append(file)

print(f"✅ Loaded {len(known_names)} known faces")

# ============ STEP 4: FACE MATCH ============
print("🧠 Matching face...")

matches = face_recognition.compare_faces(known_encodings, cnic_encoding, tolerance=0.5)
distances = face_recognition.face_distance(known_encodings, cnic_encoding)

best_match = np.argmin(distances)

if matches[best_match]:
    print("✅ MATCH FOUND:", known_names[best_match])
    print("📏 Similarity Score:", distances[best_match])
else:
    print("❌ No match found")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


🔍 Extracting text from CNIC...
📝 OCR TEXT: PAKISTAN National Identity Card ISLAMIC REPUBLIC OF PAKISTAN Name Muhammad Mudassir Zaman Ulii_} Father Name Shafqat Zaman Gender Country of Stay M Pakistan Identity Number Date of Birth 42201-2908700-1 16.04.2001 Date of Issue Date of Expiry 20.06.2019 20.06.2029 Holder 5 Signature Uljez
🆔 CNIC Number: 42201-2908700-1
🙂 Extracting face from CNIC...
📂 Loading known faces...
✅ Loaded 402 known faces
🧠 Matching face...
✅ MATCH FOUND: me.jpeg
📏 Similarity Score: 0.4840879194092022
